In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt

import tensorflow as tf

from keras import optimizers
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Get info of image
image_path = glob.glob('./Data/Picture_data/PT/' + '*jpg')[0]
image = cv2.imread(image_path)
img_height, img_weight, _ = image.shape
batch_size = 16

In [ ]:
# Turn trainingsdataset into tensorflowdataset
train_images = tf.keras.preprocessing.image_dataset_from_directory(
directory='Data/Picture_data/', 
labels='inferred', 
label_mode = 'categorical',
color_mode='rgb',
batch_size=batch_size, 
image_size = (img_height, img_weight),
shuffle=True, 
validation_split=0.2,
seed = 123,
subset="training"
)

In [ ]:
# Turn validationdataset into tensorflowdataset
validation_images = tf.keras.preprocessing.image_dataset_from_directory(
directory='Data/Picture_data/', 
labels='inferred', 
label_mode = 'categorical',
color_mode='rgb',
batch_size=2, 
image_size = (img_height, img_weight),
shuffle=True, 
validation_split=0.1,
seed = 123,
subset="validation"
)

In [ ]:
# Normalize all data
def normalize(image, label):
    image = tf.cast(image/255, tf.float32)
    return image, label

train_images = train_images.map(normalize)
validation_images = validation_images.map(normalize)

In [ ]:
# Define model

model = models.Sequential()
model.add(layers.Conv2D(filters=64, kernel_size=3, input_shape=(img_height, img_weight, 3)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
# Summary of model
model.summary()

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit the model
history = model.fit(train_images, validation_data=validation_images, epochs=1)

In [ ]:
# Presenting the results
plt.plot(history.history['accuracy'], label='accuracy')
#plt.plot(history.history['validation_accuracy'], label='validation_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='upper right')
test_loss, test_acc = model.evaluate(validation_images)


In [ ]:
# Presenting the results
print('Accuracy', test_acc)